In [3]:
import torch
from train import prepare_cifar
from model import ResNet18
import os
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

In [4]:
_, testloader = prepare_cifar()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
cudnn.benchmark = True

net_set = []
for i in range(5):
    net_set.append(ResNet18())
    net_set[i] = net_set[i].to(device)
    net_set[i] = torch.nn.DataParallel(net_set[i])
    checkpoint = torch.load("./checkpoint/ckpt_{}.pth".format(i))
    net_set[i].load_state_dict(checkpoint["net"])
    print("{} {}".format(checkpoint["acc"], checkpoint["epoch"]))

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
94.39 88
94.38 89
94.54 79
94.29 87
94.37 75


In [15]:
import copy
import numpy as np
def bad_case(net_set):
    pred_results = []
    gt = np.array([], dtype=np.int32)
    for i in range(5):
        net_set[i].eval()
        results = np.array([], dtype=np.int32)
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(testloader):
                if i == 0:
                    gt = np.concatenate((gt, targets), axis=0)
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = net_set[i](inputs)
                _, pred = outputs.max(1)
                pred = copy.deepcopy(pred.to("cpu").numpy())
                results = np.concatenate((results, pred), axis=0)
                print(batch_idx)
        print(results.shape)
        pred_results.append(results)
    
    def iou(pred1, pred2, gt):
        inter_sz = 0
        union_sz = 0
        for i in range(pred1.shape[0]):
            if gt[i] != pred1[i] or gt[i] != pred2[i]:
                union_sz += 1
                if pred1[i] == pred2[i]:
                    inter_sz += 1
        return inter_sz / union_sz
    
    def iou2(pred1, pred2, gt):
        inter_sz = 0
        union_sz = 0
        for i in range(pred1.shape[0]):
            if gt[i] != pred1[i] or gt[i] != pred2[i]:
                union_sz += 1
                if gt[i] != pred1[i] and gt[i] != pred2[i]:
                    inter_sz += 1
        return inter_sz / union_sz
    
    for i in range(5):
        for j in range(i + 1, 5):
            print(iou(pred_results[i], pred_results[j], gt), iou2(pred_results[i], pred_results[j], gt))
bad_case(net_set)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
(10000,)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
(10000,)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
(10000,)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
(10000,)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
